In [33]:
pip install fastapi "uvicorn[standard]" requests 

Note: you may need to restart the kernel to use updated packages.


In [ ]:

import requests
from fastapi import FastAPI


API_KEY = "Your API KEY" 
BASE_URL = "https://api.openweathermap.org/data/2.5/weather"

app = FastAPI()


def get_recommendation(weather_description: str, temperature: float) -> str:
    weather_description = weather_description.lower()
    if "rain" in weather_description or "storm" in weather_description:
        return "Perfect day to visit a museum or watch a movie."
    elif temperature > 25:
        return "It's hot! A great day for the beach or a swim."
    elif temperature > 15:
        return "Pleasant weather. How about a walk in the park or a bike ride?"
    elif temperature < 5:
        return "It's cold! A good time for some indoor ice skating or a cozy cafe."
    else:
        return "It's a bit chilly. A good day for a brisk walk or some shopping."


@app.get("/recommendation/{city}")
def get_activity_recommendation(city: str):
    
    request_url = f"{BASE_URL}?q={city}&appid={API_KEY}"
    response = requests.get(request_url)

    if response.status_code == 200:
        data = response.json()

        if data.get('cod') == '404':
            return {"error": f"City '{city}' not found."}

        
        weather_description = data['weather'][0]['description']
        temperature_kelvin = data['main']['temp']
        temperature_celsius = temperature_kelvin - 273.15

        
        recommendation = get_recommendation(weather_description, temperature_celsius)

        
        return {
            "city": city,
            "weather": weather_description,
            "temperature_celsius": round(temperature_celsius, 2),
            "recommendation": recommendation
        }
    else:
        return {"error": "Could not connect to the weather service."}